## Packages

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data

In [3]:
# load dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# train and test datasets are directly available, extract them accordingly
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
# no validation sets so we have to split it on our own (in train data)

# store number of validation samples, test samples into dedicated variables
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples # .num_examples returns the dataset size
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # cast as integer to prevent errors
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
# we want to scale our data so that the results are more numerically stable
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale) # .map() method allows us to apply a custom transformation (scale) to a given dataset
test_data = mnist_test.map(scale)

# shuffle the data
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# extract validation and train data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples) # .take() method takes that many samples
train_data = shuffled_train_and_validation_data.skip(num_validation_samples) # .skip() method skips that many samples

In [5]:
# set batch size (1 -> SGD, no. of samples -> mini-batch GD) (we want a relatively small batch size wrt to dataset, but reasonably high to preserve underlying dependencies)
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

# MNIST data is iterable and in 2-tuple format (as_supervised=True)
validation_inputs, validation_targets = next(iter(validation_data)) 
# iter() is the pythonic syntax for making the validation data an iterator, but does not load the data
# next() loads the next batch

## Model


In [6]:
# outline model
input_size = 784 # our data is such that each observation is 28 by 28 by 1 (rank 3 tensor)
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # .flatten() is a layers method that transforms a tensor into a vector
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # ReLu works well for this problem
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # in practice, each NN has a different optimal combination of activation functions
    tf.keras.layers.Dense(output_size, activation='softmax') # when we are creating a classifier, activation function of the output layer must result in probabilities
])

# width and depth are likely suboptimal
# fine tune to obtain better results (?)

In [7]:
# choose optimizer and loss function
# our model and optimizer expects the output shape to match the target shape in a one-hot encoded format
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# in TF2.0 there are 3 built-in variations of cross-entropy loss
# 1. binary cross-entropy 2. categorical cross-entropy 3. sparse categorical cross-entropy (equivalent to 2. but applies one-hot encoding)

In [8]:
# training
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.3389 - accuracy: 0.9022 - val_loss: 0.1807 - val_accuracy: 0.9493 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1418 - accuracy: 0.9579 - val_loss: 0.1296 - val_accuracy: 0.9612 - 1s/epoch - 2ms/step
Epoch 3/5
540/540 - 1s - loss: 0.1023 - accuracy: 0.9689 - val_loss: 0.1007 - val_accuracy: 0.9695 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0795 - accuracy: 0.9762 - val_loss: 0.0735 - val_accuracy: 0.9793 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0646 - accuracy: 0.9804 - val_loss: 0.0723 - val_accuracy: 0.9777 - 1s/epoch - 2ms/step
